In [1]:
import numpy as np
import pandas as pd


In [2]:
class MultiClassLogisticRegression:
    def __init__(self, learning_rate=0.3, num_iterations=1000, threshold=0.5):
        """
        Initialize Multi-class Logistic Regression model
        """
        self.learning_rate = learning_rate
        self.num_iterations = num_iterations
        self.threshold = threshold
        self.weights = None
        self.biases = None
        self.n_classes = None
        
    def softmax(self, z):
        """
        Compute softmax function for multi-class classification
        """
        shifted_z = z - np.max(z, axis=1, keepdims=True)
        exp_z = np.exp(shifted_z)
        return exp_z / np.sum(exp_z, axis=1, keepdims=True)
    
    def initialize_parameters(self, num_features, num_classes):
        """
        Initialize weights and biases for each class
        """
        self.n_classes = num_classes
        self.weights = np.zeros((num_features, num_classes))
        self.biases = np.zeros(num_classes)
        
    def compute_cost(self, y_true, y_pred):
        """
        Compute cross-entropy loss for multi-class classification
        """
        m = len(y_true)
        epsilon = 1e-15
        y_pred = np.clip(y_pred, epsilon, 1 - epsilon)
        return -np.sum(y_true * np.log(y_pred)) / m
    
    def one_hot_encode(self, y):
        """
        Convert class labels to one-hot encoded format
        """
        m = len(y)
        y_one_hot = np.zeros((m, self.n_classes))
        for i in range(m):
            y_one_hot[i, y[i]] = 1
        return y_one_hot
    
    def fit(self, X, y):
        """
        Train the model using gradient descent
        """
        m, n = X.shape
        self.n_classes = len(np.unique(y))
        self.initialize_parameters(n, self.n_classes)
        
        # Convert y to one-hot encoded format
        y_one_hot = self.one_hot_encode(y)
        
        # Early stopping parameters
        epsilon = 1e-5
        patience = 5
        best_cost = np.inf
        no_improvement_count = 0
        costs = []
        
        for iteration in range(self.num_iterations):
            # Forward propagation
            z = np.dot(X, self.weights) + self.biases
            y_pred = self.softmax(z)
            
            # Compute cost
            cost = self.compute_cost(y_one_hot, y_pred)
            costs.append(cost)
            
            # Early stopping check
            if cost < best_cost - epsilon:
                best_cost = cost
                no_improvement_count = 0
            else:
                no_improvement_count += 1
            
            if no_improvement_count >= patience:
                print(f"Early stopping at iteration {iteration}")
                break
            
            # Compute gradients
            dz = y_pred - y_one_hot
            dw = (1/m) * np.dot(X.T, dz)
            db = (1/m) * np.sum(dz, axis=0)
            
            # Update parameters
            self.weights -= self.learning_rate * dw
            self.biases -= self.learning_rate * db
            
            if iteration % 100 == 0:
                print(f"Iteration {iteration}: Cost = {cost:.4f}")
        
        return costs
    
    def predict_proba(self, X):
        """
        Predict class probabilities
        """
        z = np.dot(X, self.weights) + self.biases
        return self.softmax(z)
    
    def predict(self, X):
        """
        Predict class labels
        """
        probas = self.predict_proba(X)
        return np.argmax(probas, axis=1)
    
    def accuracy_score(self, y_true, y_pred):
        """
        Calculate accuracy score from scratch
        """
        return np.sum(y_true == y_pred) / len(y_true)
    
    def confusion_matrix(self, y_true, y_pred):
        """
        Compute confusion matrix from scratch
        """
        matrix = np.zeros((self.n_classes, self.n_classes), dtype=int)
        for t, p in zip(y_true, y_pred):
            matrix[t, p] += 1
        return matrix
    
    def class_metrics(self, confusion_matrix):
        """
        Compute precision, recall, and f1-score for each class
        """
        metrics = {}
        for class_idx in range(self.n_classes):
            # True Positives: diagonal elements
            tp = confusion_matrix[class_idx, class_idx]
            
            # False Positives: sum of column minus TP
            fp = np.sum(confusion_matrix[:, class_idx]) - tp
            
            # False Negatives: sum of row minus TP
            fn = np.sum(confusion_matrix[class_idx, :]) - tp
            
            # Calculate metrics
            precision = tp / (tp + fp) if (tp + fp) > 0 else 0
            recall = tp / (tp + fn) if (tp + fn) > 0 else 0
            f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
            
            metrics[f"Class {class_idx}"] = {
                "precision": precision,
                "recall": recall,
                "f1-score": f1,
                "support": np.sum(confusion_matrix[class_idx, :])
            }
        
        return metrics


In [3]:
if __name__ == "__main__":
    # Load and prepare data
    df = pd.read_csv("multi_classification_train.csv")
    
    # Separate features and target
    X = df.iloc[:, 1:-1].values  # All features except ID and Class
    y = df.iloc[:, -1].values    # Class column
    
    # Create and train model
    model = MultiClassLogisticRegression(learning_rate=0.1, num_iterations=1000)
    costs = model.fit(X, y)
    
    # Make predictions
    y_pred = model.predict(X)
    
    # Calculate metrics from scratch
    accuracy = model.accuracy_score(y, y_pred)
    conf_matrix = model.confusion_matrix(y, y_pred)
    class_metrics = model.class_metrics(conf_matrix)
    

Iteration 0: Cost = 1.6094
Early stopping at iteration 5


In [4]:
 print(f"\nTraining Accuracy: {accuracy:.4f}")


Training Accuracy: 0.5476


In [6]:
print("\nConfusion Matrix:")
print(conf_matrix)


Confusion Matrix:
[[  101     8   704  2696   531]
 [    1   563  4284  3129  3427]
 [    0     9 15523   992    94]
 [    0     4  2027  7988    45]
 [    1     9  1713  2040  2111]]


In [7]:
    print("\nDetailed Metrics by Class:")
    for class_name, metrics in class_metrics.items():
        print(f"\n{class_name}:")
        for metric_name, value in metrics.items():
            print(f"{metric_name}: {value:.4f}")


Detailed Metrics by Class:

Class 0:
precision: 0.9806
recall: 0.0250
f1-score: 0.0488
support: 4040.0000

Class 1:
precision: 0.9494
recall: 0.0494
f1-score: 0.0939
support: 11404.0000

Class 2:
precision: 0.6401
recall: 0.9341
f1-score: 0.7596
support: 16618.0000

Class 3:
precision: 0.4742
recall: 0.7937
f1-score: 0.5937
support: 10064.0000

Class 4:
precision: 0.3400
recall: 0.3594
f1-score: 0.3494
support: 5874.0000


In [8]:
# Load and prepare the test data
test_df = pd.read_csv("multi_classification_test.csv")

# Separate features and true labels
X_test = test_df.iloc[:, 1:].values  # All features except ID and Class

y_test_pred = model.predict(X_test)
y_test_pred_proba = model.predict_proba(X_test)

# Save predictions to a CSV file
output_df = pd.DataFrame({
    "ID": test_df.iloc[:, 0],  # Assuming the first column is an identifier
    "Predicted_Class": y_test_pred
})

# Optionally, add probabilities for each class
for i in range(model.n_classes):
    output_df[f"Probability_Class_{i}"] = y_test_pred_proba[:, i]

# Save to file
output_df.to_csv("predictions.csv", index=False)

print("Predictions saved to 'predictions.csv'.")


Predictions saved to 'predictions.csv'.
